In [35]:
import jqdatasdk as jq
import pandas as pd
import numpy as np
import datetime
from jqdatasdk import *
jq.auth('15661317439','Ax020629.')

获取单个股票行情数据，进行周四买，周一卖的操作，计算单次收益率，不能出现连续买卖的现象。

In [36]:
# 获取单个股票行情数据函数
def get_single_price(code,time_freq,start_date=None,end_date=None): 
    # 如果start_date=None,默认为从上市日期开始
    if start_date is None:
        # start_date = get_security_info(code).start_date
        start_date = '2023-01-01'

    # 如果end_date为None,默认为今天
    if end_date is None:
        # end_date = datetime.datetime.today()
        end_date = '2023-06-01'

    # 获取行情数据
    data = get_price(code, start_date=start_date,end_date=end_date,
                     frequency=time_freq, panel=False) 
    return data

In [37]:
# 整合信号函数(防止连续买卖)
def compose_signal(data):
    # 防止连续买
    data['buy'] = np.where((data['buy'] == 1)
                   & (data['buy'].shift(1) == 1), 0, data['buy'])
    # 防止连续卖
    data['sell'] = np.where((data['sell'] == -1)
                    & (data['sell'].shift(1) == -1), 0, data['sell'])
    # 整合买入卖出操作
    data['signal'] = data['buy'] + data['sell']
    return data    

In [38]:
# 计算单次收益率
def calculate_prof_pct(data):
    data.loc[data['signal'] != 0,'profit_pct'] = data['close'].pct_change()
    return data

In [39]:
# 周期选股（周四买，周一卖）函数
def week_period_strategy(code,time_freq,start_date=None,end_date=None):
    data = get_single_price(code,time_freq,start_date,end_date)

    # 新建周期字段
    data['weekday'] = data.index.weekday
    # 周四买入
    data['buy'] = np.where((data['weekday']==3),1,0)
    # 周一卖出
    data['sell'] = np.where((data['weekday']==0),-1,0)

    data = compose_signal(data) # 整合信号
    data = calculate_prof_pct(data) # 计算收益    
    
    return data

In [40]:
data = week_period_strategy('000001.XSHE','daily')
data.head(20)

,open,close,high,low,volume,money,weekday,buy,sell,signal,profit_pct
2023-01-03,12.89,13.44,13.52,12.74,224759667.0,2.971547e+09,1,0,0,0,NaN
2023-01-04,13.38,13.98,14.08,13.31,224304293.0,3.110729e+09,2,0,0,0,NaN
2023-01-05,14.06,14.14,14.39,14.03,170600995.0,2.417272e+09,3,1,0,1,0.011445
2023-01-06,14.16,14.27,14.37,14.14,122488383.0,1.747915e+09,4,0,0,0,NaN
2023-01-09,14.40,14.45,14.53,14.17,108343322.0,1.561368e+09,0,0,-1,-1,0.012614
2023-01-10,14.41,14.10,14.54,14.05,130035799.0,1.851094e+09,1,0,0,0,NaN
2023-01-11,14.11,14.32,14.43,14.05,85080620.0,1.217451e+09,2,0,0,0,NaN
2023-01-12,14.42,14.32,14.42,14.18,64094241.0,9.143677e+08,3,1,0,1,0.000000
2023-01-13,14.32,14.59,14.59,14.20,97221412.0,1.407181e+09,4,0,0,0,NaN
2023-01-16,14.59,14.72,14.92,14.50,159805652.0,2.356400e+09,0,0,-1,-1,0.008910
